In [1]:
import pandas as pd
df =  pd.read_csv('train_subset.csv')
df_validation = pd.read_csv('validation_subset.csv')
df_test = pd.read_csv('test_subset.csv')

In [2]:
pd.options.display.max_colwidth = 1000
df_test

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben', ' 😕 ']",...,...
1,ekaterini@gmail.com,833,"Der Klimawandel ist ein völlig normaler Prozess, den der Mensch nicht beeinflussen kann.\nChange my Mind",positive,...,...,völlig normaler Prozess,Klimawandel
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialog nicht an eine missliebige Partei vermieten. \n\nWas denken die „#Demokraten“ damit bezwecken zu wollen?,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten während Corona auf so viel verzichten"". Der Satz kommt bestimmt von denen, die während Corona auch gern auf die Maske verzichtet haben.",negative,Corona,die während Corona auch gern auf die Maske verzichtet haben.,...,...
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahnen.\nIsrael handelt moralisch richtig.,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']"
...,...,...,...,...,...,...,...,...
295,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Zivilbevölkerung im Gaza Streifen aber Kritik an Israel will niemand hören,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Israel will niemand hören']",...,...
296,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...
297,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise soviel Energie, Zeit und Geld in die Entwicklung einer hypokapitalistischen, fahrenden Kleinstadt auf dem Meer investieren. Ich könnte mir wirklich nichts vorstellen, wo diese Ressourcen besser zu gebrauchen wären! Nice one, Humanity! 💸",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity', ' 💸']",...,...
298,ekaterini@gmail.com,473,"Kleine Nachhilfe: Ein paar hunderttausend Grüne sind nicht die Mehrheit in einem Land mit über 80 Millionen Einwohner. Wir sprechen uns nochmal, wenn ihr Grünen bei der Europawahl eine Klatsche bekommen habt und hinter der AfD steht...",negative,Grüne,eine Klatsche bekommen habt,...,...


In [ ]:
import spacy
nlp = spacy.load('de_core_news_md')


In [ ]:

#ignoring neutrals as we do not consider these for NER evaluation
df = df[df['sentiment'] != 'neutral']
df_test = df_test[df_test['sentiment'] != 'neutral']
df_validation = df_validation[df_validation['sentiment'] != 'neutral']
df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

### Tokenizing the text and extracting each word along with the emoji

In [47]:
def word_extraction_spacy(text):
    #emojis = ['👏','🤔','🤡','💙','🤝','🥲','🤩','✋','😅','💛','🔥','👍','💔','🤮','💚','🤦','💪','😷','✊','🤢','😭','🙌','😣','🥺','🙁','😡','🤣','😱','🇩🇪','😏','😥','😊','🙏','😮','😵','💫','✊🏼','✊','😂','🤔','🍀','🌻','✌🏼','😉','🙃','😆','😑','🤨','🤦🏼‍♀️','🤦🏼','😠','🇺🇦','🖕🏻']
    emojis_test =['🙁','😕','🇺🇦','🤡','🇩🇪','🤣','🏆','💙','😡','🤓','😃','😆','😎','😉','🙂','✋','🍀','🙈','😁','🤝','🕊','⬆️','😂','🤣','🤦🏼','💔','👌🏻','🇵🇸','🔥','😤','🙏','👍','👏','🤷🏻','⛔️','😔','💪','😅🙃','🇮🇱','😪','🍉','🇩🇪🇨🇭🇦🇹🇹🇷','🎉','💁','🤦','😅','🤪','👏','💸','😅','✌🏽','😞','👍','🤝🏼','🇩🇪🤝🏻🇺🇦','💛','😳','👏','🌳','👍','💪','💙🇮🇱','😜','🤣','😊','🇦🇹❤️🇮🇱','🙄','😬','🙄','🌱','😢','👍💛','😂','🇩🇪🤡','🤷','💪','👍','😱','🤦🏻','🤦🏼','🇵🇸','😕','😏','😥🙏','😂','😂','😊','😠','😮','💫','✊🏼','👍🏼','🤔','😃','🌎🍀','😉','🌻','🇺🇦','😡','✌🏼','🕊️','😑','😆','🤨','🇩🇪','🤦','♂️🤦','😢','❌','💪','🇺🇸','🇮🇷','🏳️','🌈', '✌️🇵🇸♥️🇧🇷♥️🇿🇦✌️', '♂️🤡','💔','🛑','🚫!!!','👏😂','💚','🙂.','❌❌💔','🙈','🤦','🇩🇪🇮🇱🙏','🙏','🙇🏼','🔥','👍','😷','💚🔥','💚🌳','😔','😡','🥰','😂😂😂','🤡🤡🤡','🇮🇱','🙁','😔','🙌','😣','🤩','👍']
    
    tokens = []
    tokenized_input = nlp(text)
    for token in tokenized_input:
        token = str(token)
        #if token in df['negative object_spans'][i] or token in df['negative spans'][i] or token in df['positive object_spans'][i] or token in df['positive spans'][i]:
        if token.isalpha() or token in emojis_test :
            tokens.append(token)
    return tokens
    #print(len(tokens))

In [48]:
df['tokens']=df["text"].apply(lambda x : word_extraction_spacy(x))
df_validation['tokens']= df_validation['text'].apply(lambda x: word_extraction_spacy(x))
df_test['tokens'] = df_test['text'].apply(lambda x: word_extraction_spacy(x))


In [49]:
#adding a label column to the dfs
label2id = {'negative': 0, 'neutral':2, 'positive':1}
df['label'] = df['sentiment'].map(label2id)
df_test['label'] = df_test['sentiment'].map(label2id)
df_validation['label'] = df_validation['sentiment'].map(label2id)


### Creating ner tags for each word by checking whether they are present in the original spans or not

In [52]:
def ner_tags_creation_ForNEREval_new(row):
    labels = []
    for index,token in enumerate(row['tokens']):
        if token.strip() in row['negative object_spans'] or token in row['negative spans'] or token in row['positive spans'] or token in row['positive object_spans'] :
            labels.append(row['label'])
        else:
            labels.append(2)
    return labels

In [54]:
df['ner_tags'] = df.apply(ner_tags_creation_ForNEREval_new,axis=1)
df_test['ner_tags'] = df_test.apply(ner_tags_creation_ForNEREval_new,axis=1)
df_validation['ner_tags'] = df_validation.apply(ner_tags_creation_ForNEREval_new,axis=1)


In [57]:
#Dropping unnecessary columns which are not required for token classification
df.drop(columns=['user','instance_id'],inplace=True)
df_validation.drop(columns=['user','instance_id'],inplace=True)
df_test.drop(columns=['user','instance_id'],inplace=True)

In [58]:
#Inserting an id field/column
df.insert(0,'id',df.index)
df_validation.insert(0,'id',df_validation.index)
df_test.insert(0,'id',df_test.index)

In [59]:

#Saving to CSV files which can be used later for fine tuning token classification model
df.to_csv('Pre-processed_dataForAutoModelTokenClassification-trainData_new.csv',index=False)
df_validation.to_csv('Pre-processed_dataForAutoModelTokenClassification-ValidationData_new.csv', index=False)
df_test.to_csv('Pre-processed_dataForAutoModelTokenClassification-TestData_new.csv', index=False)